In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
import torchvision.datasets as datasets 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split, Subset
from CustomDataset import CustomDataset
from VanillaVAE import VanillaVAE 
# import wandb


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = 3
#38804
Z_DIM = 500
NUM_EPOCHS = 200
BATCH_SIZE = 32
LR_RATE = 3e-4
KL_COEFF_MAX = 0.0000002
PATH = "bestmodel_wandb_newer.pt"

current_epoch = 0

In [3]:
batch_size = 16
data_length = 10000
start_index = 190000
train_size = int(data_length*0.8)

dataset_val = CustomDataset("data/img_align_celeba", [(str(i).rjust(6, '0')+".jpg") for i in range(start_index, start_index + data_length+1)], transform=transforms.ToTensor(),loadToRam=True)

validation_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

0
1000


In [4]:
checkpoint = torch.load(PATH, map_location=device)

model = VanillaVAE(INPUT_DIM, Z_DIM).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [5]:
def validation(model, loss_fn):
    model.eval()
    total_reconst_loss = 0
    loss = 0
    for x in validation_loader:
        with torch.no_grad():
            x = x.to(device)
            x_reconst, _, mu, sigma = model(x)
            reconst_loss, kl_div = loss_fn(x_reconst, x, mu, sigma)['loss']
            #print("reconst_loss", reconst_loss)
            loss = loss + reconst_loss + kl_div*KL_COEFF_MAX
            total_reconst_loss = total_reconst_loss + reconst_loss
    print("Validation loss: ", loss.item()/len(validation_loader))
    print("Reconstruction loss: ", total_reconst_loss/len(validation_loader))
    

validation(model, model.loss_function)

Validation loss:  0.006826086649819026
Reconstruction loss:  tensor(0.0065)
